In [2]:
import pandas as pd

In [3]:
df = pd.read_csv("/Users/sharmiliprathap/Desktop/Portfolio 2/Data/medicine_master.csv")
df.head()

,medicine_name,strength,category
0,Metformin,500mg,Diabetes
1,Metformin,800mg,Diabetes
2,Glimepiride,1mg,Diabetes
3,Glimepiride,2mg,Diabetes
4,Sitagliptin,100mg,Diabetes


In [4]:
import requests
from bs4 import BeautifulSoup

print("Environment ready")

Environment ready


In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import time
import re
import pandas as pd

# Setup Chrome
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

# Initialize master_df
master_df = pd.DataFrame()

def scrape_1mg(url, drug_name):
    driver.get(url)
    time.sleep(5)

    # Scroll to load the cards in the inspect tool
    for i in range(2):
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(4)

    soup = BeautifulSoup(driver.page_source, "html.parser")

    # lists to hold search page data
    brands = []
    prices = []
    availability = []
    strengths = []
    product_links = []

    
    cards = soup.find_all("div", class_="VerticalProductTile__container__etaGT")

    for card in cards:
        # 1. Get the Link 
        parent_a = card.find_parent("a", href=True)
        if parent_a:
            href = parent_a['href']
            full_url = "https://www.1mg.com" + href if href.startswith('/') else href
            product_links.append(full_url)
        else:
            product_links.append(None)

        # 2. Brand Name
        brand_tag = card.find("div", class_="VerticalProductTile__header__z1Knt")
        brand_name_text = brand_tag.text.strip() if brand_tag else "Unknown"
        brands.append(brand_name_text)

        # 3. Strength 
        strength_match = re.search(r'(\d+\.?\d*)', brand_name_text)
        strengths.append(strength_match.group(1) if strength_match else "")

        # 4. Price
        price_tag = card.find("span", class_="l5Medium")
        if price_tag:
            price_match = re.search(r'(\d+\.?\d*)', price_tag.get_text())
            prices.append(price_match.group(1) if price_match else "NA")
        else:
            prices.append("NA")

        # 5. Availability
        button = card.find("button")
        availability.append("In Stock" if button and "Add to cart" in button.text else "Out of Stock")


    # Opening each individual drug page
    
    salts = []
    manufacturers = [] 
    
    for link in product_links:
        if not link:
            salts.append("NA")
            manufacturers.append("NA")
            continue
            
        try:
            driver.get(link)
            time.sleep(3) 
            
            p_soup = BeautifulSoup(driver.page_source, "html.parser")
            
            # 1. Extract Salt Composition
            salt_label = p_soup.find(string=re.compile("Salt Composition", re.I))
            if salt_label:
                container = salt_label.find_parent("div")
                salt_val = container.find_next_sibling("div")
                if salt_val:
                    salts.append(salt_val.get_text(strip=True))
                else:
                    salts.append(container.get_text().replace("Salt Composition", "").strip())
            else:
                salts.append("NA")

            # 2. Extract Manufacturer (Marketer)
            mfg_label = p_soup.find(string=re.compile("Marketer", re.I))
            if not mfg_label:
                mfg_label = p_soup.find(string=re.compile("Manufacturer", re.I))

            if mfg_label:
                mfg_container = mfg_label.find_parent("div")
                mfg_val = mfg_container.find_next_sibling("div")
                if mfg_val:
                    manufacturers.append(mfg_val.get_text(strip=True))
                else:
                    manufacturers.append(mfg_container.get_text().replace("Marketer", "").replace("Manufacturer", "").strip())
            else:
                manufacturers.append("NA")

        except:
            salts.append("Error")
            manufacturers.append("Error")

    
    global master_df

    n = len(brands)  
    df = pd.DataFrame({
        "drug_name": [drug_name] * n,
        "strength": strengths,
        "brand_name": brands,
        "salt_composition": salts,
        "manufacturer": manufacturers,
        "price": prices,
        "availability": availability,
        "website": ["1mg"] * n
    })

    master_df = pd.concat([master_df, df], ignore_index=True)


In [6]:
# Diabetic drugs scraping

scrape_1mg("https://www.1mg.com/search/all?name=metformin%20500mg", "Metformin")
scrape_1mg("https://www.1mg.com/search/all?name=metformin%20800mg", "Metformin")
scrape_1mg("https://www.1mg.com/search/all?name=Glimepiride%201mg", "Glimepiride")
scrape_1mg("https://www.1mg.com/search/all?name=Glimepiride%202mg", "Glimepiride")
scrape_1mg("https://www.1mg.com/search/all?name=Sitagliptin%20100mg", "Sitagliptin")
scrape_1mg("https://www.1mg.com/search/all?name=Vildagliptin%2050mg", "Vildagliptin")
scrape_1mg("https://www.1mg.com/search/all?name=Empagliflozin%2010mg", "Empagliflozin")
scrape_1mg("https://www.1mg.com/search/all?name=Dapaglifozine%2010mg", "Dapaglifozine")
scrape_1mg("https://www.1mg.com/search/all?name=Acarbose%2050mg", "Acarbose")
scrape_1mg("https://www.1mg.com/search/all?name=Pioglitazone%2015mg", "Pioglitazone")
master_df

,drug_name,strength,brand_name,price,availability,website
0,Metformin,500,Glycomet 500 SR Tablet,₹41.2,In Stock,1mg
1,Metformin,50,ISTamet 50mg/500mg Tablet,₹173,In Stock,1mg
2,Metformin,1,Glycomet-GP 1 Tablet PR,₹122,In Stock,1mg
3,Metformin,2,Glycomet-GP 2 Tablet PR,₹177,In Stock,1mg
4,Metformin,50,Janumet 50mg/500mg Tablet,₹345,In Stock,1mg
...,...,...,...,...,...,...
1433,Pioglitazone,5,Glibentel PM 5mg/500mg/15mg Tablet,₹24,In Stock,1mg
1434,Pioglitazone,1000,Sitanorm PM 1000mg/15mg/100mg Tablet,₹158,In Stock,1mg
1435,Pioglitazone,15,Vylda P 15mg/50mg Tablet,₹155,In Stock,1mg
1436,Pioglitazone,1,Xilia-MP 1 Tablet PR,₹135.61,In Stock,1mg


In [8]:
# Cardiac Drug Scraping

scrape_1mg("https://www.1mg.com/search/all?name=Amlodipine%205mg", "Amlodipine")
scrape_1mg("https://www.1mg.com/search/all?name=Losartan%2050mg", "Losartan")
scrape_1mg("https://www.1mg.com/search/all?name=Telmisartan%2040mg", "Telmisartan")
scrape_1mg("https://www.1mg.com/search/all?name=Atenolol%2050mg", "Atenolol")
scrape_1mg("https://www.1mg.com/search/all?name=Metoprolol%2050mg", "Metoprolol")
scrape_1mg("https://www.1mg.com/search/all?name=Atorvastatin%2010mg", "Atorvastatin")
scrape_1mg("https://www.1mg.com/search/all?name=Atorvastatin%2020mg", "Atorvastatin")
scrape_1mg("https://www.1mg.com/search/all?name=Aspirin%2075mg", "Aspirin")
scrape_1mg("https://www.1mg.com/search/all?name=Clopidogrel%2075mg", "Clopidogrel")
scrape_1mg("https://www.1mg.com/search/all?name=Enalapril%205mg", "Enalapril")
master_df

,drug_name,strength,brand_name,price,availability,website
0,Metformin,500,Glycomet 500 SR Tablet,₹41.2,In Stock,1mg
1,Metformin,50,ISTamet 50mg/500mg Tablet,₹173,In Stock,1mg
2,Metformin,1,Glycomet-GP 1 Tablet PR,₹122,In Stock,1mg
3,Metformin,2,Glycomet-GP 2 Tablet PR,₹177,In Stock,1mg
4,Metformin,50,Janumet 50mg/500mg Tablet,₹345,In Stock,1mg
...,...,...,...,...,...,...
2927,Enalapril,5,Enalet H 5mg/12.5mg Tablet,₹65.3,Out of Stock,1mg
2928,Enalapril,5,Enalaid 5 Tablet,₹32.4,Out of Stock,1mg
2929,Enalapril,12.5,Enapril HT 12.5mg/5mg Tablet,₹143,In Stock,1mg
2930,Enalapril,5,AMZEL-EL 5MG/2.5MG TABLET,₹35.16,Out of Stock,1mg


In [9]:
# Antibiotics

scrape_1mg("https://www.1mg.com/search/all?name=Amoxicillin%20500mg", "Amoxicillin")
scrape_1mg("https://www.1mg.com/search/all?name=Azithromycin%20500mg", "Azithromycin")
scrape_1mg("https://www.1mg.com/search/all?name=Ciprofloxacin%20500mg", "Ciprofloxacin")
scrape_1mg("https://www.1mg.com/search/all?name=Doxycycline%20100mg", "Doxycycline")
scrape_1mg("https://www.1mg.com/search/all?name=Cefixime%20200mg", "Cefixime")
scrape_1mg("https://www.1mg.com/search/all?name=Cefuroxime%20500mg", "Cefuroxime")
scrape_1mg("https://www.1mg.com/search/all?name=Metronidazole%20400mg", "Metronidazole")
scrape_1mg("https://www.1mg.com/search/all?name=Levofloxacin%20500mg", "Levofloxacin")
scrape_1mg("https://www.1mg.com/search/all?name=Ofloxacin%20200mg", "Ofloxacin")
scrape_1mg("https://www.1mg.com/search/all?name=Amoxiclav%20625mg", "Amoxiclav")
master_df


,drug_name,strength,brand_name,price,availability,website
0,Metformin,500,Glycomet 500 SR Tablet,₹41.2,In Stock,1mg
1,Metformin,50,ISTamet 50mg/500mg Tablet,₹173,In Stock,1mg
2,Metformin,1,Glycomet-GP 1 Tablet PR,₹122,In Stock,1mg
3,Metformin,2,Glycomet-GP 2 Tablet PR,₹177,In Stock,1mg
4,Metformin,50,Janumet 50mg/500mg Tablet,₹345,In Stock,1mg
...,...,...,...,...,...,...
4110,Ofloxacin,200,Xone O 200mg/200mg Tablet,₹317.6,In Stock,1mg
4111,Ofloxacin,,Zil-O Tablet,₹181,In Stock,1mg
4112,Ofloxacin,,Oxipod-O Tablet,₹251,In Stock,1mg
4113,Amoxiclav,875,Amoxyclav 875mg/125mg Tablet,₹516,In Stock,1mg


In [ ]:
# Pain Relief

scrape_1mg("https://www.1mg.com/search/all?name=Ibuprofen%20400mg", "Ibuprofen")
scrape_1mg("https://www.1mg.com/search/all?name=Diclofenac%2050mg", "Diclofenac")
scrape_1mg("https://www.1mg.com/search/all?name=Aceclofenac%20100mg", "Aceclofenac")
scrape_1mg("https://www.1mg.com/search/all?name=Naproxen%20500mg", "Naproxen")
scrape_1mg("https://www.1mg.com/search/all?name=Tramadol%2050mg", "Tramadol")
scrape_1mg("https://www.1mg.com/search/all?name=Etoricoxib%2060mg", "Etoricoxib")
scrape_1mg("https://www.1mg.com/search/all?name=Etoricoxib%2090mg", "Etoricoxib")
scrape_1mg("https://www.1mg.com/search/all?name=Ketorolac%2010mg", "Ketorolac")
master_df

,drug_name,strength,brand_name,price,availability,website
0,Metformin,500,Glycomet 500 SR Tablet,₹41.2,In Stock,1mg
1,Metformin,50,ISTamet 50mg/500mg Tablet,₹173,In Stock,1mg
2,Metformin,1,Glycomet-GP 1 Tablet PR,₹122,In Stock,1mg
3,Metformin,2,Glycomet-GP 2 Tablet PR,₹177,In Stock,1mg
4,Metformin,50,Janumet 50mg/500mg Tablet,₹345,In Stock,1mg
...,...,...,...,...,...,...
5649,Mefenamic acid,500,Extromef P 500mg/325mg Tablet,₹54,Out of Stock,1mg
5650,Mefenamic acid,500,Meftazid P Forte 500mg/325mg Tablet,₹45,Out of Stock,1mg
5651,Mefenamic acid,500,Meftagesic Forte 500mg/325mg Tablet,₹36.8,Out of Stock,1mg
5652,Mefenamic acid,500,Mefbrake P Forte 500mg/325mg Tablet,₹44.1,Out of Stock,1mg


In [ ]:
master_df.to_csv("raw_1mg.csv", index=False, encoding="utf-8-sig")

In [12]:
print(master_df.shape)
print(master_df.head())

(5654, 6)
   drug_name strength                 brand_name  price availability website
0  Metformin      500     Glycomet 500 SR Tablet  ₹41.2     In Stock     1mg
1  Metformin       50  ISTamet 50mg/500mg Tablet   ₹173     In Stock     1mg
2  Metformin        1    Glycomet-GP 1 Tablet PR   ₹122     In Stock     1mg
3  Metformin        2    Glycomet-GP 2 Tablet PR   ₹177     In Stock     1mg
4  Metformin       50  Janumet 50mg/500mg Tablet   ₹345     In Stock     1mg


In [13]:
driver.quit()